## 라이브러리 선언

In [90]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.pipeline import PipelinePromptTemplate

from dotenv import load_dotenv

## API_KEY 선언

In [91]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

In [92]:
# 작업을 수행할 chat 선언
chat = OpenAI(openai_api_key=OPENAI_API_KEY)

# Zero-Shot

In [93]:
full_template = """
{intro}
{input_prompt}
 """
full_prompt = PromptTemplate.from_template(full_template)

In [94]:
intro_template = """
너는 cs 담당자야 \n
주어진 text를 분석해서 긍정 리뷰인지 부정 리뷰인지 확인하려고 해 \n
주어지는 리뷰가 긍정이면 '긍정 리뷰입니다', 부정이면 '부정 리뷰압니다' 로 분류해줘
"""
intro_prompt = PromptTemplate.from_template(intro_template)

In [95]:
input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

In [96]:
prompts = [
    ("intro", intro_prompt),
    ("input_prompt", input_prompt),
]

In [97]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [98]:
# txt 형식을 dataframe으로 불러옴
with open('../data/쇼핑/naver_shopping.txt', 'r') as f:
    data = f.readlines()

data_split = [x.strip().split('\t') for x in data[0:]]
df = pd.DataFrame(data_split, columns=['score', 'review'])

In [99]:
# n개 만큼 추출하여 input으로 저장
df_sampled = df.sample(n=50, random_state=42)

formatted_prompts = [pipeline_prompt.format(
    input=row['review']) for index, row in df_sampled.iterrows()]

In [100]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

In [101]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [102]:
def extract_negative(text):
    if '부정' in text:
        return '부정'
    elif '긍정' in text:
        return '긍정'
    else:
        return None

In [103]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 84.00%


# Few-Shot 
## N=3

## Template 정의

### full_template 
- 전체 템플릿의 형식 정의

    - intro : 초입에 해당하며, 앞으로 어떤 내용이 오고 어떤 결과값을 원하는지 정의
    - example : 예시 데이터 정의
    - input_prompt : 앞의 내용을 바탕으로 결과값을 받기 위한 input 정의

In [104]:
full_template = """
{intro}
{example}
{input_prompt}
 """
full_prompt = PromptTemplate.from_template(full_template)

### intro_template 정의

In [105]:
intro_template = """
너는 cs 담당자야 \n
주어진 text를 분석해서 긍정 리뷰인지 부정 리뷰인지 확인하려고 해 \n
예시 데이터는 <text> 리뷰 </text> <tag> 긍정/부정 리뷰입니다 </tag> 형식의 xml로 주어질거야 \n
예시 데이터를 보고 주어지는 리뷰가 긍정이면 '긍정 리뷰입니다', 부정이면 '부정 리뷰압니다' 로 분류해줘
"""
intro_prompt = PromptTemplate.from_template(intro_template)

### example_template 정의
- text : 리뷰 텍스트
- tag : text에 대한 긍/부정 값

In [106]:
example_template = """ 
<text>'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요'</text>
<tag>'부정 리뷰입니다'</tag>
"""
example_prompt = PromptTemplate.from_template(example_template)

### input_template 정의

In [107]:
input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

### prompt 정의

In [108]:
prompts = [
    ("intro", intro_prompt),
    ("example", example_prompt),
    ("input_prompt", input_prompt),
]

## prompt pipeline 정의

In [109]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [111]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

## 성능 평가

In [112]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [113]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 86.00%


# Few-Shot 
## N=5

In [114]:
example_template = """ 
<text>'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜린데...'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물준사람도 받은사람도 모두 만족했어요^^'</text>
<tag>'긍정 리뷰입니다'</tag>
"""
example_prompt = PromptTemplate.from_template(example_template)

In [115]:
input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

In [116]:
prompts = [
    ("intro", intro_prompt),
    ("example", example_prompt),
    ("input_prompt", input_prompt),
]

In [117]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [119]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

In [120]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [121]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 78.00%


# Few-Shot 
## N=10

In [122]:
example_template = """ 
<text>'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜린데...'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물준사람도 받은사람도 모두 만족했어요^^'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'생각보다 너무 작아서요 나중에 먼지먹는식물이나 넣어야겠어요 물건은 흠없습니다'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'재구매 3통짼대 아주좋네요!강추'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'실망이네요 마무리처리가 너무엉성합니다'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'배송느림.. 레깅스 밑단 마무리 엉성함.. 수트화면보다 밝음..주황색같아요'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'작은 사이즈이지만 시원하고 좋아요. 설치 엄청 빨리 잡아주셨어요. 설치비가 물건값의 거의 2배인게 좀 속상하지만 넘 깔끔히 작업 해 주시고 친절히 대해주셨어요. 추천!'</text>
<tag>'긍정 리뷰입니다'</tag>

"""
example_prompt = PromptTemplate.from_template(example_template)

input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

prompts = [
    ("intro", intro_prompt),
    ("example", example_prompt),
    ("input_prompt", input_prompt),
]

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [124]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

In [125]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [126]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 86.00%


# Few-Shot
## N=20

In [127]:
example_template = """ 
<text>'아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'처음 쓸때만 좋고 쓰다보니 의자뺄때 소리 계속 나요 그래서 붙이는부직포 덧붙였는데 여전히 끌리네요 비추에요 테니스공으로 살껄 이쁜거 산다고 이거 사서 후회되요'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'가성비 괜찮습니다 바퀴가 고정된다면 별다섯개짜린데...'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'선물준사람도 받은사람도 모두 만족했어요^^'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'생각보다 너무 작아서요 나중에 먼지먹는식물이나 넣어야겠어요 물건은 흠없습니다'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'재구매 3통짼대 아주좋네요!강추'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'실망이네요 마무리처리가 너무엉성합니다'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'배송느림.. 레깅스 밑단 마무리 엉성함.. 수트화면보다 밝음..주황색같아요'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'작은 사이즈이지만 시원하고 좋아요. 설치 엄청 빨리 잡아주셨어요. 설치비가 물건값의 거의 2배인게 좀 속상하지만 넘 깔끔히 작업 해 주시고 친절히 대해주셨어요. 추천!'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'변하지않는 꾀돌이👍'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'재구매 좋아요.아이가 아주 좋아해요♡'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'매우 신선 열어놓으니까 점점 활발해짐. 황장이 가득 수율도 80퍼 이상, 신선해서 살결이 그대로 느껴짐 바로 또 먹고 싶음'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'현관문옆에 홈바식으로 사용하는데 이뻐요 한가지 단점은 아래 수납할때 기둥때문에 박스같은게 못들어가요.. ㅜ'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'공항에서 잘 수령하여 잘쓰고 있습니다'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'뽁뽁이 기사님이 엉뚱한데 두고 갔지만 하루만에 다시 와서 감사'</text>
<tag>'긍정 리뷰입니다'</tag>
<text>'가격대비 좀 부족한게 많은듯'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'안좋아요 신생아인데 잘안먹으려고 하네요ㅜ너무 조금씩나와서 빨기힘든가봐요 그린맘쓰다가 바꿔봤는데 실패ㅜ'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'재구매 이 곰팡이냄새 나는 상자에 담아주는 왜 그런건가요? 종이지관을 담아주는데 곰팡이 핀 상자에 담아주면 찝찝해서 어떻게 사용합니까? 너무하는거 아닙니까?'</text>
<tag>'부정 리뷰입니다'</tag>
<text>'사이즈가 좀 큰편이고 챙부분 마감처리와 모자옆면 마감처리가 좀 엉성하네요 가격대비 그저 그렇습니다...'</text>
<tag>'부정 리뷰입니다'</tag>

"""
example_prompt = PromptTemplate.from_template(example_template)

input_template = """ 
<text> {input} </text>
"""
input_prompt = PromptTemplate.from_template(input_template)

prompts = [
    ("intro", intro_prompt),
    ("example", example_prompt),
    ("input_prompt", input_prompt),
]

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=prompts
)

In [129]:
# 응답을 저장할 response 선언
responses = []
# formatted_prompts의 input에 대한 응답 받아오기 수행
for prompt in formatted_prompts:
    response = chat(prompt)
    responses.append(response)

In [130]:
# 응답 결과값과 실제 값을 비교하기 위해 컬럼으로 저장
df_sampled['tag'] = df_sampled['score'].apply(lambda x: '긍정' if int(x) >=3 else '부정')
df_sampled['predict'] = responses

In [131]:
# '긍부정'만 추출하여 새로운 컬럼에 저장
df_sampled['predict'] = df_sampled['predict'].apply(extract_negative)

df_sampled['is_correct'] = df_sampled['predict'] == df_sampled['tag']
# 정확도 계산
accuracy = df_sampled['is_correct'].mean()
# 결과 출력
print(f"정확도: {accuracy * 100:.2f}%")

정확도: 84.00%
